# Obtención de los Hyperparametros Optimizados para modelo de detección de YoloV8

**Importamos las librerias y modulos necesarios:**

In [13]:
from ultralytics import YOLO
from matplotlib import pyplot as plt
from PIL import Image
import optuna
import locale
import os
import json
import pandas as pd
import numpy as np
import yaml
import torch
import cv2
import random
from pathlib import Path
import time
import shutil
from matplotlib import pyplot as plt
import re

Asegurar que hay una gpu disponible

In [14]:
print("CUDA disponible:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Número de dispositivos CUDA disponibles:", torch.cuda.device_count())
    print("Nombre del dispositivo CUDA actual:", torch.cuda.get_device_name())

CUDA disponible: True
Número de dispositivos CUDA disponibles: 1
Nombre del dispositivo CUDA actual: NVIDIA GeForce RTX 3090 Ti


**Rutas del directorio donde se han guarrdad los modelos**

In [15]:
# Ruta al directorio que contiene todos los trials
ruta_resultados = "models/Final/YoloV8_n_25_fix/"

**Juntamos todas las metricas en un solo archivo**

In [16]:
# Contar el número de carpetas que siguen el patrón "trial_"
trial_folders = [folder for folder in os.listdir(ruta_resultados) if folder.startswith("trial_") and folder.split("_")[1].isdigit()]
n_trials = len(trial_folders)
print(n_trials)

# Crear una lista para almacenar todas las métricas
all_metrics = []

# Recorrer todas las carpetas de trials y leer los archivos JSON de métricas
for trial_num in range(n_trials):
    trial_dir = os.path.join(ruta_resultados, f"trial_{trial_num}")
    result_path = os.path.join(trial_dir, f"trial_{trial_num}_result.json")
    # result_path = os.path.join(trial_dir, f"metrics.json")
    if os.path.exists(result_path):
        with open(result_path, 'r') as f:
            trial_metrics = json.load(f)
            trial_metrics["trial_number"] = trial_num  # Añadir el número del trial a las métricas
            all_metrics.append(trial_metrics)

# Guardar todas las métricas en un archivo JSON
all_metrics_path = os.path.join(ruta_resultados, "all_metrics.json")
with open(all_metrics_path, 'w') as f:
    json.dump(all_metrics, f, indent=2)

print(f"Métricas de todos los trials guardadas en {all_metrics_path}")


Métricas de todos los trials guardadas en models/Final/YoloV8_n_25_fix/all_metrics.json


**Se verifica con que dataset fue entrenado el modelo**

In [17]:
# Leer el archivo all_metrics.json
with open(all_metrics_path, 'r') as f:
    all_metrics = json.load(f)

# Extraer el valor del campo "Dataset"
if all_metrics:
    data_yaml_path = all_metrics[0]["Dataset"]
else:
    data_yaml_path = None

print(f"El archivo YAML utilizado es: {data_yaml_path}")


El archivo YAML utilizado es: /home/hpc22/computer_vision_colon/TFM/Entrenamiento/datasets/Polipos265_Detectron2FIX_YoloV8/data.yaml


In [18]:
def crear_directorio(directorio):
    """Crea un directorio si no existe."""
    print(directorio)
    if not os.path.exists(directorio):
        os.makedirs(directorio)
        print(f"Directorio '{directorio}' creado.")
    else:
        print(f"El directorio '{directorio}' ya existe.")

In [19]:
def procesar_trial(trial_path):
    # Definir la ruta para la carpeta del trial y el archivo all_metrics.json
    model_path = os.path.join(trial_path, "weights/best.pt")
    
    # Cragamos el modelo     
    model_segmentation = YOLO(model_path)
    print("\n\nCargado modelo: ", model_path)
    
    # Crear carpeta para coordenadas
    output_folder = os.path.join(trial_path, "coordenadas_test")
    crear_directorio(output_folder)
    
    with open(data_yaml_path, 'r') as f:
        data_yaml = yaml.safe_load(f)

    # Obtener la ruta relativa del directorio de test
    relative_test_folder = data_yaml['test']
    
    # Imprimir la ruta de validación para verificar
    print(f"Ruta de la carpeta de test relativa: {relative_test_folder}")

    clean_test_folder = relative_test_folder.replace("../", "", 1)
    print(f"Ruta de la carpeta de test limpia: {clean_test_folder}")

    # Obtener el directorio base del archivo YAML
    yaml_base_path = Path(data_yaml_path).parent

    # Imprimir la ruta de validación para verificar
    print(f"Ruta de la carpeta YAML parent: {yaml_base_path}")

    # Resolver el camino relativo correctamente a partir de la ruta base
    test_folder = os.path.join(yaml_base_path, clean_test_folder)

    # Imprimir la ruta de validación para verificar
    print(f"Ruta de la carpeta de test: {test_folder}")

    # Procesar imágenes
    for img_path in Path(test_folder).glob('*'):
        print(f"Procesando imagen: {img_path}")

        # Realizar predicción
        results = model_segmentation(img_path)

        # Asegurarse de que results sea una lista
        if isinstance(results, list):
            # Usar el primer elemento de la lista como objeto Results
            result = results[0]
        else:
            result = results

        # Verificar el tipo y formato del objeto result
#         print(f"Tipo de resultados: {type(result)}")
#         print(f"Contenido de resultados: {result}")

        # Si result es una instancia de Results
        if hasattr(result, 'save_txt'):
            # Guardar directamente las coordenadas en un archivo .txt
            txt_path = os.path.join(output_folder, f"{img_path.stem}.txt")
            result.save_txt(txt_path, save_conf=True)  # Guarda con confianza si es necesario

            # Esperar un breve momento para asegurarse de que el archivo se haya guardado
            time.sleep(0.1)

            # Verificar y corregir coordenadas impares en el archivo guardado
            if os.path.exists(txt_path):
                with open(txt_path, 'r') as f:
                    lines = f.readlines()

                corrected_lines = []
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) % 2 == 0:
                        parts = parts[:-1]  # Eliminar el último elemento para hacer el número de coordenadas par
                    corrected_lines.append(' '.join(parts))

                # Guardar las líneas corregidas
                with open(txt_path, 'w') as f:
                    f.write('\n'.join(corrected_lines))

                print(f"Opción 1. Archivo de coordenadas guardado y corregido en: {txt_path}")
            else:
                print(f"Error: No se pudo encontrar el archivo {txt_path} después de guardarlo.")

        else:
            # Extraer resultados manualmente si no se puede usar save_txt
            detections = result.xyxy[0].cpu().numpy()  # Usar el formato adecuado

            # Contar el número de objetos detectados
            num_objects = len(detections)
            print(f"Número de objetos detectados: {num_objects}")

            if num_objects > 0:
                print("Se han detectado objetos en esta imagen:")
                for det in detections:
                    class_id, x_min, y_min, x_max, y_max, confidence = det
                    print(f"Clase: {int(class_id)}, Confianza: {confidence}, Coordenadas: [{x_min}, {y_min}, {x_max}, {y_max}]")
            else:
                print("No se detectaron objetos en esta imagen.")

            # Guardar coordenadas en archivo .txt
            txt_path = os.path.join(output_folder, f"{img_path.stem}.txt")
            with open(txt_path, 'w') as f:
                for det in detections:
                    class_id, x_min, y_min, x_max, y_max, confidence = det
                    x_center = (x_min + x_max) / 2
                    y_center = (y_min + y_max) / 2
                    width = x_max - x_min
                    height = y_max - y_min
                    f.write(f"{int(class_id)} {x_center} {y_center} {width} {height}\n")

            # Verificar y corregir coordenadas impares en el archivo guardado
            if os.path.exists(txt_path):
                with open(txt_path, 'r') as f:
                    lines = f.readlines()

                corrected_lines = []
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) % 2 == 0:
                        parts = parts[:-1]  # Eliminar el último elemento para hacer el número de coordenadas par
                    corrected_lines.append(' '.join(parts))

                # Guardar las líneas corregidas
                with open(txt_path, 'w') as f:
                    f.write('\n'.join(corrected_lines))

                print(f"Opción 2. Archivo de coordenadas guardado y corregido en: {txt_path}")
            else:
                print(f"Error: No se pudo encontrar el archivo {txt_path} después de guardarlo.")

        print("-" * 40)  # Separador para mayor claridad en la salida

In [20]:
# def borrar_coordenadas_test(trial_path):
#     """Eliminar la carpeta coordenadas_test si existe dentro de trial_path."""
#     coordenadas_test_path = os.path.join(trial_path, "coordenadas_test")
#     if os.path.exists(coordenadas_test_path) and os.path.isdir(coordenadas_test_path):
#         shutil.rmtree(coordenadas_test_path)
#         print(f"Carpeta 'coordenadas_test' eliminada en: {trial_path}")
#     else:
#         print(f"No se encontró la carpeta 'coordenadas_test' en: {trial_path}")
        
# # Iterar sobre cada carpeta trial y borrar coordenadas_test
# for folder in os.listdir(ruta_resultados):
#     trial_path = os.path.join(ruta_resultados, folder)
#     if os.path.isdir(trial_path) and folder.startswith("trial_") and folder.split("_")[1].isdigit():
#         borrar_coordenadas_test(trial_path)   

In [21]:
# Iterar sobre cada carpeta trial y procesar
for folder in os.listdir(ruta_resultados):
    trial_path = os.path.join(ruta_resultados, folder)
    if os.path.isdir(trial_path) and folder.startswith("trial_") and folder.split("_")[1].isdigit():
        procesar_trial(trial_path)

In [23]:
# Función para leer las coordenadas de YOLO
def read_yolo_mask(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    
    masks = {}
    
    for line in lines:
        data = list(map(float, line.strip().split()))
        category = int(data[0])
        coords = np.array(data[1:]).reshape(-1, 2)
        
        if category not in masks:
            masks[category] = []
        masks[category].append(coords)
    
    return masks

# Convertir coordenadas normalizadas a píxeles
def normalized_to_pixel_coords(coords, img_width, img_height):
    coords[:, 0] *= img_width
    coords[:, 1] *= img_height
    return coords

# Crear una máscara binaria para una categoría
def create_binary_mask(coords_list, img_width, img_height):
    mask = np.zeros((img_height, img_width), dtype=np.uint8)
    
    for coords in coords_list:
        coords = normalized_to_pixel_coords(coords, img_width, img_height)
        coords = coords.astype(np.int32)
        cv2.fillPoly(mask, [coords], 1)
    
    return mask

# Crear una imagen combinada con máscaras
def create_combined_image(gt_file, pred_file, img_width, img_height):
    # Leer las coordenadas
    gt_masks = read_yolo_mask(gt_file)
    pred_masks = read_yolo_mask(pred_file)

    # Crear imágenes de fondo negro
    combined_image = np.zeros((img_height, img_width, 3), dtype=np.uint8)

    # Crear máscaras binarias para cada categoría
    gt_binary_masks = {category: create_binary_mask(coords_list, img_width, img_height)
                       for category, coords_list in gt_masks.items()}
    pred_binary_masks = {category: create_binary_mask(coords_list, img_width, img_height)
                         for category, coords_list in pred_masks.items()}

    # Pintar las máscaras en la imagen combinada
    for category in gt_binary_masks:
        gt_mask = gt_binary_masks[category]
        pred_mask = pred_binary_masks.get(category, np.zeros((img_height, img_width), dtype=np.uint8))

        # Convertir las máscaras binarias a colores
        gt_colored = np.stack([gt_mask * 255, np.zeros_like(gt_mask), np.zeros_like(gt_mask)], axis=-1)  # Azul
        pred_colored = np.stack([np.zeros_like(pred_mask), np.zeros_like(pred_mask), pred_mask * 255], axis=-1)  # Rojo

        # Superponer las máscaras en la imagen combinada con transparencia
        combined_image = cv2.addWeighted(combined_image, 1, gt_colored, 0.5, 0)
        combined_image = cv2.addWeighted(combined_image, 1, pred_colored, 0.5, 0)

    return combined_image

# Extraer el número del trial del nombre de la carpeta
def extract_trial_number(trial_path):
    match = re.search(r'trial_(\d+)', os.path.basename(trial_path))
    return int(match.group(1)) if match else None

# Crear la carpeta Figuras si no existe y generar las imágenes combinadas y las imágenes visuales
def crear_figuras(trial_path, gt_files, pred_files, ground_truths, predictions):
    # Crear la carpeta de Figuras si no existe
    figuras_folder = os.path.join(trial_path, 'Figuras')
    if not os.path.exists(figuras_folder):
        os.makedirs(figuras_folder)

    # Seleccionar aleatoriamente 3 pares de archivos
    pairs = random.sample(list(gt_files & pred_files), min(3, len(gt_files & pred_files)))

    # Obtener la ruta de las imágenes originales
    dataset_test_path = os.path.dirname(ground_truths)
    dataset_test_images = os.path.join(dataset_test_path, 'images')

    for file_name in pairs:
        true_file = os.path.join(ground_truths, f"{file_name}.txt")
        pred_file = os.path.join(predictions, f"{file_name}.txt")
        image_file_path = os.path.join(dataset_test_images, f"{file_name}.jpg")  # Asumiendo que las imágenes son .jpg
        
        # Leer la imagen original
        original_image = cv2.imread(image_file_path)
        img_height, img_width = original_image.shape[:2]
        
        # Crear la imagen combinada de las máscaras
        combined_image = create_combined_image(true_file, pred_file, img_width, img_height)

        # Guardar la imagen combinada
        combined_image_file = os.path.join(figuras_folder, f"{file_name}_mask_eval.png")
        cv2.imwrite(combined_image_file, combined_image)

        # Dibujar los contornos de las máscaras en la imagen original
        gt_masks = read_yolo_mask(true_file)
        pred_masks = read_yolo_mask(pred_file)

        for category in gt_masks:
            for coords in gt_masks[category]:
                pixel_coords = normalized_to_pixel_coords(coords.copy(), img_width, img_height).astype(np.int32)
                cv2.polylines(original_image, [pixel_coords], isClosed=True, color=(255, 0, 0), thickness=2)  # Azul para ground truth

        for category in pred_masks:
            for coords in pred_masks[category]:
                pixel_coords = normalized_to_pixel_coords(coords.copy(), img_width, img_height).astype(np.int32)
                cv2.polylines(original_image, [pixel_coords], isClosed=True, color=(0, 0, 255), thickness=2)  # Rojo para predicción

        # Guardar la imagen visual
        visual_image_file = os.path.join(figuras_folder, f"{file_name}_mask_visual.png")
        cv2.imwrite(visual_image_file, original_image)

# Definir la función para procesar cada trial
def evaluar_trial(trial_path):
    # Extraer el número del trial
    trial_number = extract_trial_number(trial_path)
    if trial_number is None:
        raise ValueError(f"No se pudo extraer el número del trial de la ruta: {trial_path}")

    # Definir las rutas dentro del trial
    dataset_path = os.path.dirname(data_yaml_path)
    dataset_test_labels = os.path.join(dataset_path, 'test/labels')
    ground_truths = dataset_test_labels
    output_folder = os.path.join(trial_path, "coordenadas_test")
    predictions = output_folder

    # Umbral para binarización
    threshold = 0.5

    # Lista para almacenar los resultados
    results = []

    # Obtener una lista de los archivos de etiquetas y predicciones
    print(ground_truths)
    gt_files = {os.path.splitext(f)[0] for f in os.listdir(ground_truths) if f.endswith('.txt')}
    print(predictions)
    pred_files = {os.path.splitext(f)[0] for f in os.listdir(predictions) if f.endswith('.txt')}

    # Encontrar archivos comunes en ambas carpetas
    common_files = gt_files.intersection(pred_files)

    # Crear imágenes de las máscaras y de las visuales con contornos
    crear_figuras(trial_path, gt_files, pred_files, ground_truths, predictions)

    # Procesar cada archivo común
    for file_name in common_files:
        true_file = os.path.join(ground_truths, f"{file_name}.txt")
        pred_file = os.path.join(predictions, f"{file_name}.txt")

        # Leer y procesar las coordenadas de YOLO para ground truth
        gt_masks = read_yolo_mask(true_file)
        pred_masks = read_yolo_mask(pred_file)

        # Asignar dimensiones de imagen (reemplazar con las dimensiones reales)
        img_width, img_height = 640, 480

        # Crear máscaras binarias para cada categoría
        gt_binary_masks = {category: create_binary_mask(coords_list, img_width, img_height)
                           for category, coords_list in gt_masks.items()}
        pred_binary_masks = {category: create_binary_mask(coords_list, img_width, img_height)
                             for category, coords_list in pred_masks.items()}

        # Asegurarse de que las categorías estén presentes en ambas máscaras
        all_categories = set(gt_binary_masks.keys()).union(pred_binary_masks.keys())
        
        for category in all_categories:
            gt_mask = gt_binary_masks.get(category, np.zeros((img_height, img_width), dtype=np.uint8))
            pred_mask = pred_binary_masks.get(category, np.zeros((img_height, img_width), dtype=np.uint8))
            
            # Binarizar las máscaras
            binary_true_mask = (gt_mask > threshold).astype(int)
            binary_predicted_mask = (pred_mask > threshold).astype(int)
            
            # Calcular TP, TN, FP, FN
            TP = np.sum((binary_predicted_mask == 1) & (binary_true_mask == 1))
            FP = np.sum((binary_predicted_mask == 1) & (binary_true_mask == 0))
            TN = np.sum((binary_predicted_mask == 0) & (binary_true_mask == 0))
            FN = np.sum((binary_predicted_mask == 0) & (binary_true_mask == 1))
            
            # Guardar los resultados
            results.append([file_name, category, TP, TN, FP, FN])

    # Crear un DataFrame para visualizar los resultados
    results_df = pd.DataFrame(results, columns=['Filename', 'Category', 'TP', 'TN', 'FP', 'FN'])

    # Eliminar la columna 'Filename' antes de calcular la media
    df_for_mean = results_df.drop(columns=['Filename'])

    # Calcular las medias por categoría
    mean_results = df_for_mean.groupby('Category').mean()

    # Calcular las medias globales
    mean_TP = df_for_mean['TP'].mean()
    mean_TN = df_for_mean['TN'].mean()
    mean_FP = df_for_mean['FP'].mean()
    mean_FN = df_for_mean['FN'].mean()

    # Calcular las métricas
    precision_avg = mean_TP / (mean_TP + mean_FP) if (mean_TP + mean_FP) > 0 else 0
    recall_avg = mean_TP / (mean_TP + mean_FN) if (mean_TP + mean_FN) > 0 else 0
    f1_score_avg = (2 * precision_avg * recall_avg) / (precision_avg + recall_avg) if (precision_avg + recall_avg) > 0 else 0
    iou_avg = mean_TP / (mean_TP + mean_FP + mean_FN) if (mean_TP + mean_FP + mean_FN) > 0 else 0

    # Guardar los resultados en un archivo JSON
    metrics = {
        'mean_TP': mean_TP,
        'mean_TN': mean_TN,
        'mean_FP': mean_FP,
        'mean_FN': mean_FN,
        'precision_avg': precision_avg,
        'recall_avg': recall_avg,
        'f1_score_avg': f1_score_avg,
        'iou_avg': iou_avg
    }
    
    json_file = os.path.join(trial_path, f"trial_{trial_number}_metricas.json")
    with open(json_file, 'w') as f:
        json.dump(metrics, f, indent=4)

    print(f"\nResultados para el trial en {trial_path} han sido guardados en {json_file}")


In [ ]:
# def borrar_Figuras(trial_path):
#     """Eliminar la carpeta Figuras si existe dentro de trial_path."""
#     Figuras_path = os.path.join(trial_path, "Figuras")
#     if os.path.exists(Figuras_path) and os.path.isdir(Figuras_path):
#         shutil.rmtree(Figuras_path)
#         print(f"Carpeta 'Figuras' eliminada en: {trial_path}")
#     else:
#         print(f"No se encontró la carpeta 'Figuras' en: {trial_path}")
        
# # Iterar sobre cada carpeta trial y borrar Figuras
# for folder in os.listdir(ruta_resultados):
#     trial_path = os.path.join(ruta_resultados, folder)
#     if os.path.isdir(trial_path) and folder.startswith("trial_") and folder.split("_")[1].isdigit():
#         borrar_Figuras(trial_path)

In [25]:
# Iterar sobre cada carpeta trial y procesar
for folder in os.listdir(ruta_resultados):
    trial_path = os.path.join(ruta_resultados, folder)
    if os.path.isdir(trial_path) and folder.startswith("trial_") and folder.split("_")[1].isdigit():
        evaluar_trial(trial_path)

/home/hpc22/computer_vision_colon/TFM/Entrenamiento/datasets/Polipos265_Detectron2FIX_YoloV8/test/labels
models/Final/YoloV8_n_25_fix/trial_3/coordenadas_test

Resultados para el trial en models/Final/YoloV8_n_25_fix/trial_3 han sido guardados en models/Final/YoloV8_n_25_fix/trial_3/trial_3_metricas.json
/home/hpc22/computer_vision_colon/TFM/Entrenamiento/datasets/Polipos265_Detectron2FIX_YoloV8/test/labels
models/Final/YoloV8_n_25_fix/trial_21/coordenadas_test

Resultados para el trial en models/Final/YoloV8_n_25_fix/trial_21 han sido guardados en models/Final/YoloV8_n_25_fix/trial_21/trial_21_metricas.json
/home/hpc22/computer_vision_colon/TFM/Entrenamiento/datasets/Polipos265_Detectron2FIX_YoloV8/test/labels
models/Final/YoloV8_n_25_fix/trial_2/coordenadas_test

Resultados para el trial en models/Final/YoloV8_n_25_fix/trial_2 han sido guardados en models/Final/YoloV8_n_25_fix/trial_2/trial_2_metricas.json
/home/hpc22/computer_vision_colon/TFM/Entrenamiento/datasets/Polipos265_Detect

**Juntamos todos los json de las metricas**

In [9]:
# Contar el número de carpetas que siguen el patrón "trial_"
trial_folders = [folder for folder in os.listdir(ruta_resultados) if folder.startswith("trial_") and folder.split("_")[1].isdigit()]
n_trials = len(trial_folders)

# Crear una lista para almacenar todas las métricas
all_metrics = []

# Recorrer todas las carpetas de trials y leer los archivos JSON de métricas
for trial_num in range(n_trials):
    trial_dir = os.path.join(ruta_resultados, f"trial_{trial_num}")
    metrics_path = os.path.join(trial_dir, f"trial_{trial_num}_metricas.json")
    if os.path.exists(metrics_path):
        with open(metrics_path, 'r') as f:
            trial_metrics = json.load(f)
            trial_metrics["trial_number"] = trial_num  # Añadir el número del trial a las métricas
            all_metrics.append(trial_metrics)

# Guardar todas las métricas en un archivo JSON
evaluacion_path = os.path.join(ruta_resultados, "evaluacion.json")
with open(evaluacion_path, 'w') as f:
    json.dump(all_metrics, f, indent=2)

print(f"Métricas de todos los trials guardadas en {evaluacion_path}")

Métricas de todos los trials guardadas en models/YoloV8_n_25_Fix_OLD/evaluacion.json


In [10]:
# Leer el archivo JSON
with open(evaluacion_path, 'r') as f:
    metrics_data = json.load(f)

# Definir pesos para cada métrica (puedes ajustar los pesos según tu criterio)
weights = {
    "precision_avg": 0.25,
    "recall_avg": 0.25,
    "f1_score_avg": 0.25,
    "iou_avg": 0.25
}

# Crear una lista con los trials y sus medias ponderadas
trials_with_scores = []

for trial in metrics_data:
    precision = trial.get("precision_avg", 0)
    recall = trial.get("recall_avg", 0)
    f1_score = trial.get("f1_score_avg", 0)
    iou = trial.get("iou_avg", 0)
    
    # Calcular la media ponderada
    weighted_score = (
        weights["precision_avg"] * precision +
        weights["recall_avg"] * recall +
        weights["f1_score_avg"] * f1_score +
        weights["iou_avg"] * iou
    )
    
    # Añadir a la lista
    trials_with_scores.append({
        "trial_number": trial["trial_number"],
        "weighted_score": weighted_score
    })

# Ordenar los trials por la media ponderada de mayor a menor
sorted_trials = sorted(trials_with_scores, key=lambda x: x["weighted_score"], reverse=True)

# Mostrar la lista ordenada
for trial in sorted_trials:
    print(f"Trial {trial['trial_number']}: Weighted Score = {trial['weighted_score']}")

# Si quieres guardar el resultado en un archivo JSON
sorted_trials_path = os.path.join(os.path.dirname(evaluacion_path), "sorted_trials.json")
with open(sorted_trials_path, 'w') as f:
    json.dump(sorted_trials, f, indent=2)

print(f"Lista de trials ordenados guardada en {sorted_trials_path}")


Trial 19: Weighted Score = 0.8828438653400437
Trial 5: Weighted Score = 0.8608503618690455
Trial 4: Weighted Score = 0.8557497784360885
Trial 22: Weighted Score = 0.8498506759543574
Trial 14: Weighted Score = 0.8361514229151755
Trial 1: Weighted Score = 0.8347559648312743
Trial 15: Weighted Score = 0.8330633407576643
Trial 9: Weighted Score = 0.8250452055593901
Trial 18: Weighted Score = 0.8247714676312701
Trial 10: Weighted Score = 0.8240136782275047
Trial 7: Weighted Score = 0.821220675957069
Trial 12: Weighted Score = 0.8195839043911846
Trial 24: Weighted Score = 0.816776069062496
Trial 2: Weighted Score = 0.8081135172297598
Trial 21: Weighted Score = 0.7966791673431348
Trial 13: Weighted Score = 0.7896223074062703
Trial 6: Weighted Score = 0.7882793404156524
Trial 3: Weighted Score = 0.781403573227101
Trial 20: Weighted Score = 0.7786397291135185
Trial 23: Weighted Score = 0.7696108270367885
Trial 8: Weighted Score = 0.7513854050945894
Trial 0: Weighted Score = 0.7512410610846881
T

**Creamos  un json final con toda la información**

In [12]:
# Leer los archivos JSON
with open(evaluacion_path, 'r') as f:
    evaluacion_data = json.load(f)

with open(sorted_trials_path, 'r') as f:
    sorted_trials_data = json.load(f)

with open(all_metrics_path, 'r') as f:
    all_metrics_data = json.load(f)

# Crear un diccionario para acceder rápidamente a la información de sorted_trials
sorted_trials_dict = {trial['trial_number']: trial['weighted_score'] for trial in sorted_trials_data}

# Crear un diccionario para acceder rápidamente a la información de all_metrics
all_metrics_dict = {trial['Trial']: trial for trial in all_metrics_data}

# Obtener la información común del primer elemento de all_metrics_data
if all_metrics_data:
    common_info = {
        "Dataset": all_metrics_data[0].get("Dataset", ""),
        "Feedback": all_metrics_data[0].get("Feedback", ""),
        "Pre-Trained_Weights": all_metrics_data[0].get("Pre-Trained_Weights", "")
    }
else:
    common_info = {
        "Dataset": "",
        "Feedback": "",
        "Pre-Trained_Weights": ""
    }

# Crear el archivo de salida
resultados = []

# Ordenar los trial_numbers basados en sorted_trials_dict
sorted_trial_numbers = sorted(sorted_trials_dict.keys())

# Combinar la información
for trial_number in sorted_trial_numbers:
    # Buscar la información de evaluación y métricas
    evaluation_info = next((item for item in evaluacion_data if item["trial_number"] == trial_number), {})
    metrics_info = all_metrics_dict.get(trial_number, {})
    
    # Crear el objeto para el trial actual
    trial_info = {
        "trial_number": trial_number,
        "hyperparameters": metrics_info.get("hyperparameters", {}),
        "metrics": metrics_info.get("metrics", {}),
        "evaluation": evaluation_info,
        "weighted_score": sorted_trials_dict[trial_number]
    }
    
    # Añadir la información común y la información del trial a la lista de resultados
    resultados.append({**common_info, **trial_info})

# Guardar el resultado en el archivo Resultados.json
resultados_path = os.path.join(os.path.dirname(trial_dir), "Resultados.json")
with open(resultados_path, 'w') as f:
    json.dump(resultados, f, indent=2)


**Resultados ordenados por las mejores metricas**

In [ ]:
# Leer los archivos JSON
with open(evaluacion_path, 'r') as f:
    evaluacion_data = json.load(f)

with open(sorted_trials_path, 'r') as f:
    sorted_trials_data = json.load(f)

with open(all_metrics_path, 'r') as f:
    all_metrics_data = json.load(f)

with open(inference_times_path, 'r') as f:
    inference_times_data = json.load(f)

# Crear un diccionario para acceder rápidamente a la información de sorted_trials
sorted_trials_dict = {trial['trial_number']: trial['weighted_score'] for trial in sorted_trials_data}

# Crear un diccionario para acceder rápidamente a la información de all_metrics
all_metrics_dict = {trial['Trial']: trial for trial in all_metrics_data}

# Crear un diccionario para acceder rápidamente a la información de inference_times
inference_times_dict = {trial['trial_number']: trial['inference_time'] for trial in inference_times_data}

# Obtener la información común del primer elemento de all_metrics_data
if all_metrics_data:
    common_info = {
        "Dataset": all_metrics_data[0].get("Dataset", ""),
        "Feedback": all_metrics_data[0].get("Feedback", ""),
        "Pre-Trained_Weights": all_metrics_data[0].get("Pre-Trained_Weights", "")
    }
else:
    common_info = {
        "Dataset": "",
        "Feedback": "",
        "Pre-Trained_Weights": ""
    }

# Crear el archivo de salida
resultados_ordenados = []

# Extraer los trial_numbers en el orden exacto en el que aparecen en sorted_trials_data
sorted_trial_numbers = [trial['trial_number'] for trial in sorted_trials_data]

# Combinar la información
for trial_number in sorted_trial_numbers:
    # Buscar la información de evaluación y métricas
    evaluation_info = next((item for item in evaluacion_data if item["trial_number"] == trial_number), {})
    metrics_info = all_metrics_dict.get(trial_number, {})
    inference_time = inference_times_dict.get(trial_number, 0)  # Obtener el tiempo de inferencia

    # Crear el objeto para el trial actual
    trial_info = {
        "trial_number": trial_number,
        "hyperparameters": metrics_info.get("hyperparameters", {}),
        "metrics": metrics_info.get("metrics", {}),
        "evaluation": evaluation_info,
        "weighted_score": sorted_trials_dict[trial_number],
        "inference_time": inference_time  # Añadir el tiempo de inferencia
    }

    # Añadir la información común y la información del trial a la lista de resultados
    resultados_ordenados.append({**common_info, **trial_info})

# Guardar el resultado en el archivo con un nombre dinámico
resultados_path = os.path.join(os.path.dirname(trial_dir), "Resultados_ordenados_metricas.json")
with open(resultados_path, 'w') as f:
    json.dump(resultados_ordenados, f, indent=2)

print(f"Archivo Resultados_ordenados_metricas.json guardado en {resultados_path}")


**Creamos un criterio por el que obtenemos el mejor modelo de estos trials, segun el teimpo de inferencia**

In [ ]:
import json
import os

# Leer los archivos JSON
with open(evaluacion_path, 'r') as f:
    evaluacion_data = json.load(f)

with open(sorted_trials_path, 'r') as f:
    sorted_trials_data = json.load(f)

with open(all_metrics_path, 'r') as f:
    all_metrics_data = json.load(f)

with open(inference_times_path, 'r') as f:
    inference_times_data = json.load(f)

# Crear un diccionario para acceder rápidamente a la información de sorted_trials
sorted_trials_dict = {trial['trial_number']: trial['weighted_score'] for trial in sorted_trials_data}

# Crear un diccionario para acceder rápidamente a la información de all_metrics
all_metrics_dict = {trial['Trial']: trial for trial in all_metrics_data}

# Crear un diccionario para acceder rápidamente a la información de inference_times
inference_times_dict = {trial['trial_number']: trial['inference_time'] for trial in inference_times_data}

# Obtener la información común del primer elemento de all_metrics_data
if all_metrics_data:
    common_info = {
        "Dataset": all_metrics_data[0].get("Dataset", ""),
        "Feedback": all_metrics_data[0].get("Feedback", ""),
        "Pre-Trained_Weights": all_metrics_data[0].get("Pre-Trained_Weights", "")
    }
else:
    common_info = {
        "Dataset": "",
        "Feedback": "",
        "Pre-Trained_Weights": ""
    }

# Crear el archivo de salida
resultados_ordenados = []

# Extraer los trial_numbers en el orden exacto en el que aparecen en sorted_trials_data
sorted_trial_numbers = [trial['trial_number'] for trial in sorted_trials_data]

# Combinar la información
for trial_number in sorted_trial_numbers:
    # Buscar la información de evaluación y métricas
    evaluation_info = next((item for item in evaluacion_data if item["trial_number"] == trial_number), {})
    metrics_info = all_metrics_dict.get(trial_number, {})
    inference_time = inference_times_dict.get(trial_number, 0)  # Obtener el tiempo de inferencia

    # Crear el objeto para el trial actual
    trial_info = {
        "trial_number": trial_number,
        "hyperparameters": metrics_info.get("hyperparameters", {}),
        "metrics": metrics_info.get("metrics", {}),
        "evaluation": evaluation_info,
        "weighted_score": sorted_trials_dict[trial_number],
        "inference_time": inference_time  # Añadir el tiempo de inferencia
    }

    # Añadir la información común y la información del trial a la lista de resultados
    resultados_ordenados.append({**common_info, **trial_info})

# Ordenar los resultados por tiempo de inferencia (menor a mayor)
resultados_ordenados.sort(key=lambda x: x["inference_time"])

# Guardar el resultado en el archivo con un nombre dinámico
resultados_path = os.path.join(os.path.dirname(trial_dir), "Resultados_ordenados_time.json")
with open(resultados_path, 'w') as f:
    json.dump(resultados_ordenados, f, indent=2)

print(f"Archivo Resultados_ordenados_metricas.json guardado en {resultados_path}")
